In [15]:
import pandas as pd
import requests
from io import StringIO

#On s'occupe d'abord de nettoyer les données sur le nombre de validation par jour à partir des données annuelles
url = "http://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false"
response = requests.get(url, verify=False)
df = pd.read_csv(StringIO(response.text), sep=';', header=None)
df.columns = ["ID", "Type", "Station", "Traffic", "Col4", "Col5", "Col6", "Col7", "Col8", "City", "Col10"]
subset_df = df[["Traffic", "Station"]]
subset_df["Traffic"], subset_df["Station"] = subset_df["Station"].copy(), subset_df["Traffic"].copy()
subset_df.columns = ["Station", "Traffic"]
subset_df = subset_df.iloc[1:].reset_index(drop=True)
# Créer un nouveau tableau avec la somme du trafic pour chaque station
sum_traffic_df = subset_df.groupby("Station")["Traffic"].sum().reset_index()
sum_traffic_df["Traffic"] = pd.to_numeric(sum_traffic_df["Traffic"], errors='coerce')
# Diviser le trafic par 365 pour chaque station pour avoir le trafic par jour
sum_traffic_df["Traffic_Per_Day"] = (sum_traffic_df["Traffic"] / 365).astype(int)


#On s'occupe ensuite de nettoyer les données sur la répartition de l'affluence sur une journée
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre (1).csv'
données_validation= pd.read_csv(url)
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['LIBELLE_ARRET', 'TRNC_HORR_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations', aggfunc='mean').reset_index()

#On fusionne les données des deux tableaux pour avoir le nombre de validation sur chaque plage horaire
merged_df = pd.merge(sum_traffic_df, donnees_organisees, left_on='Station', right_on='LIBELLE_ARRET')
columns_to_multiply = donnees_organisees.columns[1:]
resultat = merged_df.copy()
resultat[columns_to_multiply] = resultat[columns_to_multiply].multiply(resultat['Traffic_Per_Day'], axis=0)
resultat[columns_to_multiply] = resultat[columns_to_multiply].div(100)
resultat = resultat.iloc[:, 2:]
numeric_columns = resultat.select_dtypes(include='number').columns
resultat[numeric_columns] = resultat[numeric_columns].applymap(lambda x: int(x) if pd.notna(x) else x)
numeric_columns = resultat.select_dtypes(include='number').columns

# Créer un nouveau tableau avec l'affluence divisée par minute pour chaque plage horaire
affluence_par_minute_df = resultat.copy()
affluence_par_minute_df[numeric_columns] = affluence_par_minute_df[numeric_columns].div(60)
# Supprimer la colonne 'ND'
affluence_par_minute = affluence_par_minute_df.drop(columns=['ND'])
# Supprimer la première colonne
affluence_par_minute = affluence_par_minute.drop(affluence_par_minute.columns[0], axis=1)

# Afficher le nouveau tableau
print(affluence_par_minute)


C:\Users\kelly\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.ratp.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\kelly\AppData\Local\Temp\ipykernel_18624\2276804303.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Traffic"], subset_df["Station"] = subset_df["Station"].copy(), subset_df["Traffic"].copy()


      LIBELLE_ARRET     0H-1H    10H-11H    11H-12H    12H-13H    13H-14H  \
0          ABBESSES  1.316667   2.616667   3.300000   3.916667   4.150000   
1            ALESIA  1.533333   6.600000   7.533333   9.366667   9.650000   
2      ALMA-MARCEAU  3.150000   2.800000   3.766667   6.000000   5.816667   
3    ANATOLE FRANCE  0.633333   5.316667   5.916667   6.516667   6.366667   
4            ANTONY  0.533333  10.050000  11.083333  12.466667  11.633333   
..              ...       ...        ...        ...        ...        ...   
209        VILLIERS  1.966667   7.183333   8.733333  11.283333  10.566667   
210       VINCENNES  2.500000  11.283333  13.116667  15.216667  14.950000   
211     VOLONTAIRES  0.600000   4.400000   4.866667   5.483333   5.283333   
212        VOLTAIRE  2.583333   7.350000   8.366667   9.400000   9.950000   
213          WAGRAM  0.516667   3.500000   4.083333   4.800000   4.783333   

       14H-15H    15H-16H    16H-17H    17H-18H  ...   22H-23H    23H-0H  \

In [30]:
def nombre_personnes_entrant(horaire, station):
    # Convertir l'horaire en minutes
    horaire_en_minutes = int(horaire.split('.')[0]) * 60 + int(horaire.split('.')[1])

    # Trouver la plage horaire correspondante dans le tableau
    plage_horaire = None
    for col in affluence_par_minute.columns[1:]:
        debut, fin = map(int, col.replace('H', '').split('-'))
        debut_en_minutes = debut * 60
        fin_en_minutes = fin * 60 if fin != 0 else 24 * 60  # Si la fin est 0H, utiliser 24H

        # Vérifier si l'horaire est dans la plage horaire actuelle
        if debut_en_minutes <= horaire_en_minutes < fin_en_minutes:
            plage_horaire = col
            break

    # Si la plage horaire est trouvée, rechercher la station en utilisant une correspondance partielle
    if plage_horaire:
        matching_station = affluence_par_minute['LIBELLE_ARRET'].str.contains(station, case=False)
        if matching_station.any():
            return affluence_par_minute.loc[matching_station, plage_horaire].values[0]

    # Retourner None si la plage horaire ou la station n'est pas trouvée
    return None

# Exemple d'utilisation
horaire_entree = "10.23"
station_entree = "ALESIA"
nombre_entrant = nombre_personnes_entrant(horaire_entree, station_entree)
print(f"Nombre de personnes entrant à {horaire_entree} à la station {station_entree}: {nombre_entrant}")


Nombre de personnes entrant à 10.23 à la station ALESIA: 6.6
